In [1]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb

#use a class to define your input and output in order to have mutable variables
class Mspa_process_IO:
    def __init__(self):
        #set up your inputs
        self.foreground_connectivity = None
        self.edge_width = None
        self.transition_core = True
        self.separate_feature = True
        self.statistics = True
        
        #set up output
        
mspa_process_io = Mspa_process_IO()

In [2]:
#some messages diplayed by the process
MSPA_MESSAGE = 'click on "Run process" to launch the process on your sepal account'
NO_MAP = 'No map have been provided, compute the GFC analysis first'
NO_AOI = 'No aoi have been provided, select one in step 1'
NO_INPUT ='Missing input for the process, complete all fields'

In [3]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "mspa_result_widget"
title = "Results"
mspa_results = wf.Tile(id_, title, inputs=['No results to display yet'])

In [4]:
import ipyvuetify as v

#create the output alert 
mspa_process_output = wf.OutputWidget(MSPA_MESSAGE)

#create the button that start your process
mspa_process_btn = wf.ProcessBtn('Run mspa analysis')

#create the widgets following ipyvuetify requirements 
connectivity = [4, 8] #using text input to transform it into 'confXX'
mspa_connectivity = v.Select(
    items=connectivity, 
    label='Foreground connectivity', 
    v_model=None
)
mspa_edge_slider = v.Slider(
    label= 'Edge width', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=0
)
mspa_transition = v.Switch(label="Transition Core - Loop/Bridge", v_model=True)
mspa_feature = v.Switch(label="Separate internal from external features", v_model=True)
mspa_statistics = v.Switch(label="Compute statistics", v_model=True)


mspa_process_inputs = [
    mspa_connectivity, 
    mspa_edge_slider, 
    mspa_transition, 
    mspa_feature, 
    mspa_statistics
]

#bind the widget to the inputs
wb.bind(mspa_connectivity, mspa_process_io, 'foreground_connectivity', mspa_process_output)
wb.bind(mspa_edge_slider, mspa_process_io, 'edge_width', mspa_process_output)
wb.bind(mspa_transition, mspa_process_io, 'transition_core', mspa_process_output)
wb.bind(mspa_feature, mspa_process_io, 'separate_feature', mspa_process_output)
wb.bind(mspa_statistics, mspa_process_io, 'statistics', mspa_process_output)


#create a process tile
id_ = "mspa_process_widget"
title = 'Create fragmentation map'

mspa_process = wf.Tile(
    id_, 
    title, 
    btn=mspa_process_btn, 
    inputs=mspa_process_inputs, 
    output=mspa_process_output
)

In [5]:
#bind the button to the process by writing a custom function
from scripts import run
from sepal_ui.scripts import utils
from functools import partial

def process_start(widget, event, data, output):
    
    #in order to write the results
    global mspa_results
    
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #load inputs
    #asset = getattr(gfc_aoi_IO, 'assetId')
    #threshold = getattr(gfc_process_io, 'threshold')
    #clip_map = getattr(gfc_process_io, clip_map)
    foreground_connectivity = getattr(mspa_process_io, 'foreground_connectivity')
    edge_width = getattr(mspa_process_io,'edge_width')
    transition_core = getattr(mspa_process_io, 'transition_core')
    separate_feature = getattr(mspa_process_io, 'separate_feature')
    statistics = getattr(mspa_process_io, 'statistics')
    
    #debug
    asset = 'users/bornToBeAlive/aoi_PU'
    threshold = 30
    clip_map = '/home/prambaud/gfc_wrapper_results/gfc/PU_30_gfc_map.tif'
    
    #check inputs 
    if not asset: 
        utils.displayIO(output, NO_AOI, alert_type='error')
        utils.toggleLoading(widget)
        return 
    
    if not (clip_map and threshold): 
        utils.displayIO(output, NO_MAP, alert_type='error')
        utils.toggleLoading(widget)
        return
    
    if not (foreground_connectivity or edge_width): 
        utils.displayIO(output, NO_INPUT, alert_type='error')
        utils.toggleLoading(widget)
        return
    
    #launch any process you want
    run.mspaAnalysis(
        clip_map=clip_map, 
        assetId=asset, 
        threshold=threshold, 
        foreground_connectivity=foreground_connectivity, 
        edge_width=edge_width, 
        transition_core=transition_core, 
        separate_feature=separate_feature, 
        statistics=statistics,
        output=output
    )
    
    #utils.displayIO(output, 'on a cliqué', alert_type='success')
    
    #toggle the loading button
    utils.toggleLoading(widget)

mspa_process_btn.on_event('click', partial(process_start, output=mspa_process_output))

In [6]:
#this tiles will only be displayed if you launch voila from this file 
mspa_process

Layout(align_center=True, children=[Card(children=[Html(children=['Create fragmentation map'], tag='h2'), Flex…

mspa masked map already ready
chmod 755 /home/prambaud/gfc_wrapper_results/tmp/mspa/mspa_lin64


FileNotFoundError: [Errno 2] No such file or directory: '/home/prambaud/gfc_wrapper_results/tmp/mspa/output/input_8_1_1_1_1_stat.txt'

In [7]:
#this tiles will only be displayed if you launch voila from this file 
mspa_results

Layout(align_center=True, children=[Card(children=[Html(children=['Results'], tag='h2'), Flex(children=[Layout…

In [8]:
#print(getattr(mspa_process_io, 'foreground_connectivity'))
#print(getattr(mspa_process_io,'edge_width'))
#print(getattr(mspa_process_io, 'transition_core'))
#print(getattr(mspa_process_io, 'separate_feature'))
#print(getattr(mspa_process_io, 'statistics'))